<a href="https://colab.research.google.com/github/mkaran02/Automated-Energy-Audit-Report-Generator/blob/main/auditreport.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from fpdf import FPDF


In [32]:
import pandas as pd

def load_energy_data(file_path="/content/AEP_hourly.csv"):
    df = pd.read_csv(file_path)  # Change to pd.read_excel(file_path) for Excel files
    df["Datetime"] = pd.to_datetime(df["Datetime"])
    df = df.set_index("Datetime")

    # Rename consumption column if needed
    if "AEP_MW" not in df.columns:
        df.rename(columns={df.columns[1]: "AEP_MW"}, inplace=True)

    return df

# Example usage
df = load_energy_data()
print(df.head())


                      AEP_MW
Datetime                    
2004-12-31 01:00:00  13478.0
2004-12-31 02:00:00  12865.0
2004-12-31 03:00:00  12577.0
2004-12-31 04:00:00  12517.0
2004-12-31 05:00:00  12670.0


In [33]:
def analyze_energy(df):
    report = {}
    report["Total Consumption (MW)"] = df["AEP_MW"].sum()
    report["Average Daily Consumption (MW)"] = df["AEP_MW"].resample("D").sum().mean()
    report["Peak Consumption (MW)"] = df["AEP_MW"].max()
    report["Peak Time"] = df["AEP_MW"].idxmax()

    return report

# Analyze energy usage
report_data = analyze_energy(df)
print(report_data)


{'Total Consumption (MW)': 1879672527.0, 'Average Daily Consumption (MW)': 371844.21899109794, 'Peak Consumption (MW)': 25695.0, 'Peak Time': Timestamp('2008-10-20 14:00:00')}


In [34]:
def detect_anomalies(df):
    threshold = df["AEP_MW"].mean() + 2 * df["AEP_MW"].std()
    anomalies = df[df["AEP_MW"] > threshold]
    return anomalies

# Detect energy spikes
anomalies = detect_anomalies(df)
print(anomalies)


                      AEP_MW
Datetime                    
2004-12-20 07:00:00  21616.0
2004-12-20 08:00:00  22554.0
2004-12-20 09:00:00  22577.0
2004-12-20 10:00:00  22321.0
2004-12-20 11:00:00  22055.0
...                      ...
2018-01-03 00:00:00  20799.0
2018-01-01 19:00:00  20925.0
2018-01-01 20:00:00  21089.0
2018-01-01 21:00:00  20999.0
2018-01-01 22:00:00  20820.0

[3989 rows x 1 columns]


In [35]:
def plot_energy_trends(df):
    # Get the last 30 days of data
    last_date = df.index.max()  # Find the latest date in the dataset
    last_30_days = last_date - pd.Timedelta(days=30)  # Calculate the date 30 days ago

    # Filter the dataframe for the last 30 days
    last_month = df.loc[df.index >= last_30_days]

    # Check if enough data is available
    if last_month.empty:
        print("Not enough data for the last 30 days. Plotting full dataset.")
        last_month = df  # Fallback to full dataset

    # Plot energy consumption for the last 30 days
    plt.figure(figsize=(10,5))
    sns.lineplot(x=last_month.index, y=last_month["AEP_MW"], color="blue", label="Last 30 Days Energy Consumption")
    plt.xlabel("Time")
    plt.ylabel("Energy (MW)")
    plt.title("Energy Consumption Trends - Last 30 Days")
    plt.legend()
    plt.savefig("energy_trend_last_30_days.png")
    plt.close()

# Generate the last 1-month trend chart
plot_energy_trends(df)


In [36]:
class EnergyAuditReport(FPDF):
    def header(self):
        self.set_font("Arial", "B", 16)
        self.cell(200, 10, "Automated Energy Audit Report", ln=True, align="C")
        self.ln(10)

    def footer(self):
        self.set_y(-15)
        self.set_font("Arial", "I", 10)
        self.cell(0, 10, f"Page {self.page_no()}", align="C")

def generate_pdf(report_data, anomalies):
    pdf = EnergyAuditReport()
    pdf.add_page()

    # Summary Section
    pdf.set_font("Arial", "", 12)
    pdf.cell(200, 10, "Energy Consumption Summary:", ln=True)
    for key, value in report_data.items():
        pdf.cell(200, 10, f"{key}: {value}", ln=True)

    # Anomalies Section
    pdf.ln(10)
    pdf.cell(200, 10, "Detected Anomalies (High Consumption Periods):", ln=True)
    for index, row in anomalies.iterrows():
        pdf.cell(200, 10, f"{index}: {row['AEP_MW']} MW", ln=True)

    # Attach Trend Chart for Last 30 Days
    pdf.ln(10)
    pdf.cell(200, 10, "Energy Consumption Trend (Last 30 Days):", ln=True)
    pdf.image("energy_trend_last_30_days.png", x=30, w=150)

    # Save PDF
    pdf.output("energy_audit_report.pdf")

# Generate and save the report
generate_pdf(report_data, anomalies)


In [37]:
if __name__ == "__main__":
    file_path = "/content/AEP_hourly.csv"  # Update with actual file path

    # Load data
    df = load_energy_data(file_path)

    # Analyze data
    report_data = analyze_energy(df)
    anomalies = detect_anomalies(df)

    # Generate visualization
    plot_energy_trends(df)

    # Generate PDF report
    generate_pdf(report_data, anomalies)

    print("Energy audit report generated: energy_audit_report.pdf")


Energy audit report generated: energy_audit_report.pdf


In [38]:
from google.colab import files
files.download("energy_audit_report.pdf")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>